In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import glob
import scanpy as sc
from sig_utils import sanitize, extract_cell_id_from_fasta

# Single-cell signatures

In [8]:
pwd

'/home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks'

In [9]:
PYTHON = '/home/olga/miniconda3/envs/immune-evolution/bin/python'
PWD = '/home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks'
SIG2KMER = f"{PWD}/sig2kmer.py"
sig2kmer_template = f"{PYTHON} {SIG2KMER} " + r"--quiet {moltype_flags} --output-sequences {output_fasta} --output-kmers {output_kmer_csv} {query_sig} {input_fasta}"
sig2kmer_template

'/home/olga/miniconda3/envs/immune-evolution/bin/python /home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks/sig2kmer.py --quiet {moltype_flags} --output-sequences {output_fasta} --output-kmers {output_kmer_csv} {query_sig} {input_fasta}'

In [10]:
fasta_type_to_sketch_ids = {
    "nucleotides": ["alphabet-DNA__ksize-21__scaled-10"],
    "peptides": [
        "alphabet-dayhoff__ksize-51__scaled-10",
        "alphabet-protein__ksize-30__scaled-10",
    ],
}

sketch_id_to_flags = {
    "alphabet-DNA__ksize-21__scaled-10": "--dna --ksize 21",
    "alphabet-dayhoff__ksize-51__scaled-10": "--dayhoff --no-dna --ksize 51 --input-is-protein",
    "alphabet-protein__ksize-30__scaled-10": "--protein --no-dna --ksize 30 --input-is-protein",
}

In [11]:

def clean_fasta_name(basename, strings_to_remove=["__aligned", '__possorted_genome_bam', '_possorted_genome_bam', '__unaligned']):
    new_name = None
    for to_remove in strings_to_remove:
        if new_name is None:
            # First time --> take original basename
            new_name = basename.replace(to_remove, '')
        else:
            new_name = new_name.replace(to_remove, '')
            
    new_name = new_name.split('_coding_reads')[0].strip("_")
    return new_name



# Per-celltype database signatures for diagnostic k-mers

In [1]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/

total 10K
drwxr-xr-x 20 olga czb 4.0K Apr 16 09:10 .
drwxr-xr-x  7 olga czb 4.0K Mar  5 12:02 ..
drwxr-xr-x  4 olga czb 4.0K Feb 28 10:19 0--fastas
drwxr-xr-x  5 olga czb 4.0K Apr 16 18:09 1--single-cell-sigs
drwxr-xr-x  5 olga czb 4.0K Feb 28 10:58 2--merged-celltype-sigs
drwxr-xr-x  5 olga czb 4.0K Mar 26 16:23 2--merged-celltype-sigs--min-kmer-count
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:08 2--merged-celltype-sigs--min-kmer-count--10-n-cells
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:35 2--merged-celltype-sigs--min-kmer-count--10-percent
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:04 2--merged-celltype-sigs--min-kmer-count--2-n-cells
drwxr-xr-x  5 olga czb 4.0K Mar 26 18:50 2--merged-celltype-sigs--min-kmer-count--20-percent
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:06 2--merged-celltype-sigs--min-kmer-count--5-n-cells
drwxr-xr-x  5 olga czb 4.0K Mar 26 19:33 2--merged-celltype-sigs--min-kmer-count--5-percent
drwxr-xr-x  5 olga czb 4.0K Mar 26 18:52 2--merged-celltype-sigs--min-kmer-count--50-perce

In [4]:
! ls -lha /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5-celltype-kmers

total 2.5K
drwxr-xr-x  5 olga czb 4.0K Mar 23 15:53 .
drwxr-xr-x 20 olga czb 4.0K Apr 16 09:10 ..
drwxr-xr-x  3 olga czb 4.0K Mar 25 15:46 alphabet-DNA__ksize-21__scaled-10
drwxr-xr-x  3 olga czb 4.0K Mar 30 19:26 alphabet-dayhoff__ksize-51__scaled-10
drwxr-xr-x  4 olga czb 4.0K Mar 18 16:12 fastas


## Read one2one 5had

In [20]:

one2one = sc.read(
    os.path.join(
        "/home/olga/data_sm/immune-evolution/h5ads/",
        "human_lemur_mouse_subset_w_shared_one2one_orthologs_human_lemur_mouse_bat.h5ad",
    )
)
one2one.obs["cell_id"] = (
    one2one.obs["channel"].str.split("_10X").str[0].astype(str).str.upper() + "__" + one2one.obs["cell_barcode"].astype(str)
)
one2one.obs = one2one.obs.reset_index().set_index("cell_id")
one2one.obs.head()

,index,age,cell_barcode,cell_ontology_class,cell_ontology_id,channel,free_annotation,individual,mouse.id,patient,...,compartment_group,compartment_narrow,compartment_broad,compartment_broad_narrow,compartment_species,compartment_narrow_species,n_counts,log_counts,sqrt_counts,n_genes
cell_id,,,,,,,,,,,,,,,,,,,,,
P2_1__AAACCTGAGAAACCAT,P2_1_AAACCTGAGAAACCAT-human,46y,AAACCTGAGAAACCAT,capillary endothelial cell,NaN,P2_1,Capillary Aerocyte,patient_2,NaN,2.0,...,endothelial,endothelial: Capillary Aerocyte,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,1894.0,7.546974,43.520111,1067
P2_1__AAATGCCAGATGAGAG,P2_1_AAATGCCAGATGAGAG-human,46y,AAATGCCAGATGAGAG,capillary endothelial cell,NaN,P2_1,Capillary Aerocyte,patient_2,NaN,2.0,...,endothelial,endothelial: Capillary Aerocyte,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,2891.0,7.969704,53.768021,1343
P2_1__AACACGTTCGATCCCT,P2_1_AACACGTTCGATCCCT-human,46y,AACACGTTCGATCCCT,capillary endothelial cell,NaN,P2_1,Capillary Aerocyte,patient_2,NaN,2.0,...,endothelial,endothelial: Capillary Aerocyte,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,1465.0,7.290293,38.275318,917
P2_1__AACACGTTCGCACTCT,P2_1_AACACGTTCGCACTCT-human,46y,AACACGTTCGCACTCT,capillary endothelial cell,NaN,P2_1,Capillary Aerocyte,patient_2,NaN,2.0,...,endothelial,endothelial: Capillary Aerocyte,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,2127.0,7.662938,46.119411,1068
P2_1__AACCATGCAGCTCGCA,P2_1_AACCATGCAGCTCGCA-human,46y,AACCATGCAGCTCGCA,capillary endothelial cell,NaN,P2_1,Capillary Aerocyte,patient_2,NaN,2.0,...,endothelial,endothelial: Capillary Aerocyte,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,1620.0,7.390799,40.249222,932


## Set shared celltypes

In [21]:
SHARED_CELLTYPES = ['Alveolar Epithelial Type 2',
 'B cell',
 'Capillary',
 'Dendritic',
 'Fibroblast',
 'Macrophage',
 'Monocyte',
 'Natural Killer T cell',
 'Smooth Muscle and Myofibroblast',
 'T cell']
len(SHARED_CELLTYPES)

10

In [22]:
sanitized_to_celltype = {sanitize(x): x for x in SHARED_CELLTYPES}
sanitized_to_celltype

{'alveolar_epithelial_type_2': 'Alveolar Epithelial Type 2',
 'b_cell': 'B cell',
 'capillary': 'Capillary',
 'dendritic': 'Dendritic',
 'fibroblast': 'Fibroblast',
 'macrophage': 'Macrophage',
 'monocyte': 'Monocyte',
 'natural_killer_t_cell': 'Natural Killer T cell',
 'smooth_muscle_and_myofibroblast': 'Smooth Muscle and Myofibroblast',
 't_cell': 'T cell'}

In [23]:
mouse = one2one[(one2one.obs['species'] == "Mouse") & one2one.obs.broad_group.isin(SHARED_CELLTYPES)]
mouse

/home/olga/miniconda3/envs/immune-evolution/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 19825 × 6327
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'mouse.id', 'patient', 'possibly_contaminated_barcode', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'n_counts', 'log_counts', 'sqrt_counts', 'n_genes'
    var: 'mouse_lemur__gene_name_x', 'mouse__gene_name_x', 'gene_ids-lemur', 'n_cells-mouse', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mouse_lemur__gene_name_y', 'mouse__gene_name_y', 'bat__gene_name'
    uns: 'age_colors', 'broad_group_colors', 'cell_ontology_class_colors', 'compartment_group_colors', 'individual_colors', 'narrow_group_colors', 'neighbors', 'pca', 'sex_colors', 'species_batch_colors', '

In [24]:
species_folder

'/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/'

In [57]:
fasta_base = os.path.join(
    "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse", "0--fastas"
)


celltype_fasta_base = os.path.join(
    "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse", "1--celltype-fastas"
)
! mkdir -p $celltype_fasta_base


species_folder  = '/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/'

# Add "--t" to glob command for train and test data
for sig_base in glob.glob(
    "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--*-percent"
):

    suffix = os.path.basename(sig_base).split("3--")[-1]
    output_base = os.path.join(species_folder, f"5--celltype-kmers--{suffix}")
    if not os.path.exists(output_base):
        os.makedirs(output_base)
    print(sig_base)
    print(suffix)

    #     fasta_output_base = os.path.join(output_base, '')

    command_txt = os.path.join(output_base, "sig2kmer_commands.txt")

    with open(command_txt, "w") as f:
        for fasta_type, sketch_ids in fasta_type_to_sketch_ids.items():

            input_fasta_glob = os.path.join(fasta_base, fasta_type, "*")
            #             input_fastas = glob.iglob(os.path.join(fasta_base, fasta_type, '*'))
            for sketch_id in sketch_ids:
                moltype_flags = sketch_id_to_flags[sketch_id]
                sketch_id_sig_folder = os.path.join(sig_base, sketch_id)

                sketch_id_output_base = os.path.join(output_base, sketch_id)
                fasta_output_base = os.path.join(sketch_id_output_base, "fastas")
                csv_output_base = os.path.join(sketch_id_output_base, "csvs")

                if not os.path.exists(csv_output_base):
                    os.makedirs(fasta_output_base)
                    os.makedirs(csv_output_base)
                sketch_id_celltype_sigs = glob.iglob(
                    os.path.join(sketch_id_sig_folder, "*.sig")
                )
                for celltype_sig in sketch_id_celltype_sigs:
                    celltype_sanitized = os.path.basename(celltype_sig).split(".sig")[0]
                    output_fasta = os.path.join(
                        fasta_output_base, f"{celltype_sanitized}.fasta"
                    )
                    output_kmer_csv = os.path.join(
                        csv_output_base, f"{celltype_sanitized}.csv"
                    )
                    
                    # 53 bytes is the size of the CSV header
                    if os.path.exists(output_kmer_csv) and os.path.getsize(output_kmer_csv) > 53:
                        continue
                    
                    celltype_name = sanitized_to_celltype[celltype_sanitized]

                    celltype_cell_ids = set(
                        mouse.obs.query("broad_group == @celltype_name").index
                    )

                    input_fasta_filenames = [
                        x
                        for x in glob.iglob(input_fasta_glob)
                        if extract_cell_id_from_fasta(x, double_aligned=True).upper() in celltype_cell_ids
                    ]
                    
                    
                    celltype_moltype_fasta_dir = os.path.join(celltype_fasta_base, celltype_sanitized, fasta_type)
                    if not os.path.exists(celltype_moltype_fasta_dir):
                        os.makedirs(celltype_moltype_fasta_dir)

                    
                    for filename in input_fasta_filenames:
                        destination = os.path.join(celltype_moltype_fasta_dir, os.path.basename(filename))
                        if not os.path.exists(destination):
                            os.symlink(filename, destination)
                    

                    command = sig2kmer_template.format(
                        moltype_flags=moltype_flags,
                        output_fasta=output_fasta,
                        output_kmer_csv=output_kmer_csv,
                        query_sig=celltype_sig,
                        input_fasta=os.path.join(celltype_moltype_fasta_dir, '*'),
                    )
                    f.write(command + "\n")
    #                     break

    print(f"parallel --progress --eta --jobs 30 < {command_txt}")
    ! wc -l $command_txt
    print("---")
#     ! head -n 1 $command_txt
#     print("---")

/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent
merged-celltype-remove-common-kmers--min-kmer-count--5-percent
parallel --progress --eta --jobs 30 < /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/sig2kmer_commands.txt
30 /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/sig2kmer_commands.txt
---
/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent
merged-celltype-remove-common-kmers--min-kmer-count--10-percent
parallel --progress --eta --jobs 30 < /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--10-percent/sig2kmer_commands.txt
20 /home/

In [56]:
! tail -n 5 $command_txt

/home/olga/miniconda3/envs/immune-evolution/bin/python /home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks/sig2kmer.py --quiet --protein --no-dna --ksize 30 --input-is-protein --output-sequences /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--10-percent/alphabet-protein__ksize-30__scaled-10/fastas/monocyte.fasta --output-kmers /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--10-percent/alphabet-protein__ksize-30__scaled-10/csvs/monocyte.csv /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent/alphabet-protein__ksize-30__scaled-10/monocyte.sig /home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/1--celltype-fastas/monocyte/peptides/*
/home/olga/miniconda3/envs/immune-evolution/bin/python /ho

In [37]:
# celltype_cell_ids

In [42]:
input_fasta_glob

'/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas/peptides/*'

In [46]:
for x in glob.iglob('/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/0--fastas/nucleotides/10*'):
    print(extract_cell_id_from_fasta(x, double_aligned=True))
    break

In [43]:
# celltype_cell_ids

In [29]:
%debug

> /home/olga/code/immune-evolution--olgabot/analyze-kmermaid-bladder/notebooks/sig_utils.py(605)extract_channel_alignment_barcode()
    603     else:
    604         # "__aligned__" or "__unaligned__"
--> 605         channel, alignment_status, barcode, suffix = basename.split("__")
    606     return channel.strip("_"), alignment_status.strip("_"), barcode.strip('_')
    607 



ipdb>  dir()


['basename', 'double_aligned', 'singlecell_fasta']


ipdb>  basename


'MACA_21m_F_LUNG_55__aligned__aligned__GAAGCAGCATGTCTCC__coding_reads_nucleotides.fasta'


ipdb>  q
